In [1]:
#singlet state
import qiskit

from qiskit import (QuantumRegister,
                    ClassicalRegister,
                    QuantumCircuit,
                    execute,
                    BasicAer,
                    Aer,
                    IBMQ)
from qiskit.visualization import plot_state_paulivec, plot_state_hinton
from qiskit.quantum_info import state_fidelity
from qiskit.providers.aer import noise
import numpy as np
import time
from copy import deepcopy

# Tomography functions
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
import qiskit.ignis.mitigation.measurement as mc

q = QuantumRegister(2)
simulator = Aer.get_backend("qasm_simulator")
circuit = QuantumCircuit(q)
circuit.x(q[0])
circuit.x(q[1])
circuit.h(q[1])
circuit.cx(q[1],q[0])
job = qiskit.execute(circuit, Aer.get_backend('statevector_simulator'))
psi_sing = job.result().get_statevector(circuit)
print(psi_sing)
backend = BasicAer.get_backend('unitary_simulator') 
result = execute(circuit, backend).result()
psi  = result.get_unitary(circuit, decimals=3)
print(psi)

# Generate circuits and run on simulator
t = time.time()
# Generate the state tomography circuits. Only pass in the 
# registers we want to measure (in this case 1 and 2)



noise_model = noise.NoiseModel()
for qi in range(6):
    read_err = noise.errors.readout_error.ReadoutError([[0.75, 0.25],[0.9,0.1]])
    noise_model.add_readout_error(read_err,[qi])


meas_calibs, state_labels = mc.complete_meas_cal(qubit_list=[0,1])

backend = Aer.get_backend('qasm_simulator')
job_cal = qiskit.execute(meas_calibs, backend=backend, shots=15000, noise_model=noise_model)
job_tomo = qiskit.execute(qst_sing, backend=backend, shots=15000, noise_model=noise_model)
meas_fitter = mc.CompleteMeasFitter(job_cal.result(),state_labels)
tomo_sing = StateTomographyFitter(job_tomo.result(), qst_sing)

rho_sing = tomo_sing.fit()
rhos_sing
print(rho_sing)

[ 0.        +0.j  0.70710678+0.j -0.70710678+0.j  0.        +0.j]
[[ 0.   +0.j  0.707+0.j  0.   +0.j  0.707+0.j]
 [ 0.707+0.j  0.   +0.j  0.707+0.j  0.   +0.j]
 [-0.707+0.j  0.   +0.j  0.707+0.j  0.   +0.j]
 [ 0.   +0.j -0.707+0.j  0.   +0.j  0.707+0.j]]


NameError: name 'qst_sing' is not defined

In [3]:
#singlet state
import qiskit

from qiskit import (QuantumRegister,
                    ClassicalRegister,
                    QuantumCircuit,
                    execute,
                    BasicAer,
                    Aer,
                    IBMQ)
from qiskit.visualization import plot_state_paulivec, plot_state_hinton
from qiskit.quantum_info import state_fidelity
from qiskit.providers.aer import noise
import numpy as np
import time
from copy import deepcopy

from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import QuantumError, ReadoutError
from qiskit.providers.aer.noise.errors import pauli_error
from qiskit.providers.aer.noise.errors import depolarizing_error
from qiskit.providers.aer.noise.errors import thermal_relaxation_error
from qiskit.providers.aer.noise.errors import phase_damping_error
from qiskit.providers.aer.noise.errors import amplitude_damping_error
from qiskit.providers.aer.noise.errors import phase_amplitude_damping_error
# Tomography functions
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
import qiskit.ignis.mitigation.measurement as mc
pi=3.141592652
q = QuantumRegister(2)
c = ClassicalRegister(2)
simulator = Aer.get_backend("qasm_simulator")
circuit = QuantumCircuit(q,c)
circuit.x(q[0])
circuit.x(q[1])
circuit.h(q[1])
circuit.cx(q[1],q[0])

circuit.measure(q[0],c[0])
circuit.measure(q[1],c[1])

# Generate circuits and run on simulator
t = time.time()
# Generate the state tomography circuits. Only pass in the 
# registers we want to measure (in this case 1 and 2)



noise_model1 = noise.NoiseModel()
for qi in range(2):
    read_err = noise.errors.readout_error.ReadoutError([[0.95, 0.05], [0.1,  0.9 ]])
    noise_model.add_readout_error(read_err,[qi])

noise_model2 = noise.NoiseModel()
for qi in range(2):
    error = depolarizing_error(0.05, 1)
    noise_model2.add_all_qubit_quantum_error(error, ['u1', 'u2', 'u3'])



noise_model3 = noise.NoiseModel()
for qi in range(2):
    p_gate1 = 0.05
    error_gate1 = pauli_error([('X',p_gate1), ('I', 1 - p_gate1)])
    error_gate2 = error_gate1.tensor(error_gate1)
    noise_model3.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3"])
    noise_model3.add_all_qubit_quantum_error(error_gate2, ["cx"])


T1s = np.random.normal(50e3, 10e3, 4) # Sampled from normal distribution mean 50 microsec
T2s = np.random.normal(70e3, 10e3, 4)  # Sampled from normal distribution mean 50 microsec

# Truncate random T2s <= T1s
T2s = np.array([min(T2s[j], 2 * T1s[j]) for j in range(4)])

# Instruction times (in nanoseconds)
time_u1 = 0   # virtual gate
time_u2 = 50  # (single X90 pulse)
time_u3 = 100 # (two X90 pulses)
time_cx = 300
time_reset = 1000  # 1 microsecond
time_measure = 1000 # 1 microsecond

# QuantumError objects
errors_reset = [thermal_relaxation_error(t1, t2, time_reset)
                for t1, t2 in zip(T1s, T2s)]
errors_measure = [thermal_relaxation_error(t1, t2, time_measure)
                  for t1, t2 in zip(T1s, T2s)]
errors_u1  = [thermal_relaxation_error(t1, t2, time_u1)
              for t1, t2 in zip(T1s, T2s)]
errors_u2  = [thermal_relaxation_error(t1, t2, time_u2)
              for t1, t2 in zip(T1s, T2s)]
errors_u3  = [thermal_relaxation_error(t1, t2, time_u3)
              for t1, t2 in zip(T1s, T2s)]
errors_cx = [[thermal_relaxation_error(t1a, t2a, time_cx).expand(
             thermal_relaxation_error(t1b, t2b, time_cx))
              for t1a, t2a in zip(T1s, T2s)]
               for t1b, t2b in zip(T1s, T2s)]

# Add errors to noise model
noise_model4 = NoiseModel()
for j in range(4):
    noise_model4.add_quantum_error(errors_reset[j], "reset", [j])
    noise_model4.add_quantum_error(errors_measure[j], "measure", [j])
    noise_model4.add_quantum_error(errors_u1[j], "u1", [j])
    noise_model4.add_quantum_error(errors_u2[j], "u2", [j])
    noise_model4.add_quantum_error(errors_u3[j], "u3", [j])
    for k in range(4):
        noise_model4.add_quantum_error(errors_cx[j][k], "cx", [j, k])
        
noise_model5 = noise.NoiseModel()
for qi in range(2):
    p_error = 0.05
    bit_flip = pauli_error([('Z', p_error), ('I', 1 - p_error)])
    noise_model5.add_all_qubit_quantum_error(bit_flip, ["u1", "u2", "u3", "id"])



noise_model6 = noise.NoiseModel()
for qi in range(2):
    prob = 0.05
    phase = phase_damping_error(prob, canonical_kraus=True)
    noise_model5.add_all_qubit_quantum_error(phase, ["u1", "u2", "u3", "id"])

noise_model7 = noise.NoiseModel()
for qi in range(2):
    amp_damp = amplitude_damping_error(prob, excited_state_population=0, canonical_kraus=True)
    noise_model5.add_all_qubit_quantum_error(amp_damp, ["u1", "u2", "u3", "id"])
    
noise_model8 = noise.NoiseModel()
for qi in range(2):
    phase_amp = phase_amplitude_damping_error(prob, prob, excited_state_population=0, canonical_kraus=True)
    noise_model5.add_all_qubit_quantum_error(phase_amp, ["u1", "u2", "u3", "id"])

backend = Aer.get_backend('qasm_simulator')
job_tomo = qiskit.execute(circuit, backend=backend, shots=1000)
result = job_tomo.result()
counts = result.get_counts(circuit)
print(counts, "ideal simulation")

backend = Aer.get_backend('qasm_simulator')
job_tomo = qiskit.execute(circuit, backend=backend, shots=1000, noise_model=noise_model1)
result = job_tomo.result()
counts = result.get_counts(circuit)
print(counts, "readout error")

backend = Aer.get_backend('qasm_simulator')
job_tomo = qiskit.execute(circuit, backend=backend, shots=1000, noise_model=noise_model2)
result = job_tomo.result()
counts = result.get_counts(circuit)
print(counts, "depolarization error")

backend = Aer.get_backend('qasm_simulator')
job_tomo = qiskit.execute(circuit, backend=backend, shots=1000, noise_model=noise_model3)
result = job_tomo.result()
counts = result.get_counts(circuit)
print(counts, "gate error")

backend = Aer.get_backend('qasm_simulator')
job_tomo = qiskit.execute(circuit, backend=backend, shots=1000, noise_model=noise_model4)
result = job_tomo.result()
counts = result.get_counts(circuit)
print(counts, "thermal relaxation error")

backend = Aer.get_backend('qasm_simulator')
job_tomo = qiskit.execute(circuit, backend=backend, shots=1000, noise_model=noise_model5)
result = job_tomo.result()
counts = result.get_counts(circuit)
print(counts, "bit flip error")

backend = Aer.get_backend('qasm_simulator')
job_tomo = qiskit.execute(circuit, backend=backend, shots=1000, noise_model=noise_model6)
result = job_tomo.result()
counts = result.get_counts(circuit)
print(counts, "phase damping error")

backend = Aer.get_backend('qasm_simulator')
job_tomo = qiskit.execute(circuit, backend=backend, shots=1000, noise_model=noise_model7)
result = job_tomo.result()
counts = result.get_counts(circuit)
print(counts, "amplitude damping error")

backend = Aer.get_backend('qasm_simulator')
job_tomo = qiskit.execute(circuit, backend=backend, shots=1000, noise_model=noise_model8)
result = job_tomo.result()
counts = result.get_counts(circuit)
print(counts, "amplitude+phase damping error")






{'10': 494, '01': 506} ideal simulation
{'10': 528, '01': 472} readout error
{'10': 495, '01': 505} depolarization error
{'11': 83, '10': 432, '00': 83, '01': 402} gate error
{'10': 492, '00': 27, '01': 481} thermal relaxation error
{'10': 516, '01': 484} bit flip error
{'10': 514, '01': 486} phase damping error
{'10': 470, '01': 530} amplitude damping error
{'10': 487, '01': 513} amplitude+phase damping error
